In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("Gemini API key setup complete.")
except Exception as e:
    print(f"Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets.")

✅ Gemini API key setup complete.


In [ ]:
# importing component from ADK

from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("ADK components imported successfully.")

✅ ADK components imported successfully.


<img width="800" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/multi-agent-team.png" alt="Multi-agent Team" />

In [ ]:
# defining a research agent whose job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings", # The result will be stored in the session state with this key.
)

print("research_agent created.")

✅ research_agent created.


In [ ]:
# another agent, summarizer agent, its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model="gemini-2.5-flash-lite",
    # modified instruction to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("summarizer_agent created.")

✅ summarizer_agent created.


In [ ]:
# a root coordinator that operates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model="gemini-2.5-flash-lite",
    # This instruction tells the root agent HOW to use its tools/ the sub-agents.
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # sub-agents are made callable by wraping them as tools as 'AgentTool'
    tools=[
        AgentTool(research_agent),
        AgentTool(summarizer_agent)
    ],
)

print("root_agent created.")

✅ root_agent created.


In [17]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("What are the latest advancements in LLM transformers and what do they mean for AI?")


 ### Created new session: debug_session_id

User > What are the latest advancements in LLM transformers and what do they mean for AI?


ResearchCoordinator > The latest advancements in LLM transformers are centered around their foundational self-attention mechanism, which allows for parallel processing and has dramatically improved AI efficiency and accuracy. This has led to revolutions in Natural Language Processing (NLP) with models like BERT and GPT, and has expanded into computer vision with models like ViT, as well as multimodal AI capable of processing text, images, and audio, exemplified by DALL-E.

These advancements have resulted in highly scalable models, and current research is focused on enhancing their ability to handle long-range context and improving overall efficiency. Furthermore, the development of accessible pre-trained and open-weight models is democratizing AI, moving it towards becoming a general-purpose reasoning engine with significant potential across numerous industries.
